Test making an agent straight from OpenAI API

In [1]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentExecutor, create_openai_tools_agent

load_dotenv()

OPENAI_SECRET_KEY = os.getenv("OPENAI_SECRET_KEY")

Make a custom tool which writes an entry to a diary.
Reference: https://platform.openai.com/docs/assistants/tools/function-calling/quickstart

In [2]:
import datetime
from typing import Optional

In [3]:
class DiaryEntry:

    def __init__(self, date: datetime.date, text: str):
        self.date = date
        self.text = text

    def __str__(self):
        return self.text

class Diary:

    def __init__(self):

        self.entries = {} # {datetime: [entry, ..]}

    def write_entry(self, entry):

        self.entries[entry.date] = self.entries.get(entry.date, []) + [entry]

    def get_entries(self, date):

        return self.entries.get(date, [])

In [14]:
DIARY = Diary()
def write_to_diary(text: str, date: datetime.date = datetime.date.today()):
    """
    Writes the text into the diary in the page of the corresponding date.
    Writes to the page for today if date is not provided.
    """
    DIARY.write_entry(
        DiaryEntry(
            date = date,
            text = text
        )
    )
    return True

def get_diary_entries(date: datetime.date = datetime.date.today()):
    """
    Returns the entries in the diary on the specified date.
    Returns the entries today of date is not provided.
    """
    print(date)
    print(type(date))

    entries = DIARY.get_entries(date)
    entries_str = "\n".join([e.text for e in entries])

    return entries_str

In [40]:
from textwrap import dedent
import json
from openai import OpenAI

client = OpenAI(api_key = OPENAI_SECRET_KEY)

In [12]:
# NOTE: 'datetime.date' does not work as type
    # just parse from string

ASSISTANT_TOOLS_DEFINITIONS = [
    {
        "type": "function",
        "function": {
            "name": "write_to_diary",
            "description": dedent('''
                Writes the text into the diary in the page of the corresponding date.
                Writes to the page for today if date is not provided.
                '''),
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to be written into the diary.",
                    },
                    "date": {
                        "type": "string",
                        "description": "The date (YYYY-MM-DD) in the diary wherein the text will be written. Defaults to today if not provided.",
                    },
                },
                "required": ["text"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_diary_entries",
            "description": dedent('''
                Returns the entries in the diary on the specified date.
                Returns the entries today of date is not provided.
                '''),
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "The date (YYYY-MM-DD) in the diary from where to get entries. Defaults to today if not provided.",
                    },
                },
                "required": [],
            },
        }
    },
]

In [18]:
SYSTEM_PROMPT = '''
You are a helpful assistant.
When asked to write something to the human's diary, use the write-to-diary-tool.
Unless he explicitly says to write it in his diary, confirm it first.
When asked to retrieve something from the human's diary, use the get-diary-entries-tool.
'''.strip()

In [19]:
assistant = client.beta.assistants.create(
    instructions = SYSTEM_PROMPT,
    model = "gpt-4o",
    tools = ASSISTANT_TOOLS_DEFINITIONS,
)

In [50]:
# I guess this is the equivalent of agent executor

def run_thread(thread, assistant):
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        return messages
    else:
        print(run.status)
        pass

    # Define the list to store tool outputs
    tool_outputs = []

    # Loop through each tool in the required action section
    if run.required_action:
        for tool in run.required_action.submit_tool_outputs.tool_calls:

            tool_args = json.loads(tool.function.arguments)

            if tool.function.name == "write_to_diary":
                text = tool_args["text"]
                date_str = tool_args.get("date", None)

                if date_str is None:
                    date = datetime.date.today()
                else:
                    date = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()

                output = write_to_diary(text, date)

                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": str(output),
                })
                
            elif tool.function.name == "get_diary_entries":
                date_str = tool_args.get("date", None)

                if date_str is None:
                    date = datetime.date.today()
                else:
                    date = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()

                output = get_diary_entries(date)

                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": output,
                })

    # Submit all tool outputs at once after collecting them in a list
    if tool_outputs:
        try:
            run = client.beta.threads.runs.submit_tool_outputs_and_poll(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
            print("Tool outputs submitted successfully.")
        except Exception as e:
            print("Failed to submit tool outputs:", e)
    else:
        print("No tool outputs to submit.")

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        return messages
    else:
        print(run.status)
        pass

In [51]:
DIARY = Diary()

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="write in my diary that i had a tough day today.",
)
messages = run_thread(thread, assistant)
print(messages.data[0].content[0].text.value)

Just to confirm, would you like me to write in your diary that you had a tough day today?


In [52]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="yes. write it to June 14, 2024",
)
messages = run_thread(thread, assistant)
print(messages.data[0].content[0].text.value)

requires_action
Tool outputs submitted successfully.
I've written in your diary that you had a tough day on June 14, 2024.


In [54]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="also write that traffic was bad on the same date",
)
messages = run_thread(thread, assistant)
print(messages.data[0].content[0].text.value)

requires_action
Tool outputs submitted successfully.
I've added to your diary that traffic was bad on June 14, 2024.


In [55]:
get_diary_entries(datetime.date(2024,6,14))

2024-06-14
<class 'datetime.date'>


'I had a tough day today.\nTraffic was bad.'

In [56]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what's written in my diary on june 14, 2024?",
)
messages = run_thread(thread, assistant)
print(messages.data[0].content[0].text.value)

requires_action
2024-06-14
<class 'datetime.date'>
Tool outputs submitted successfully.
Your diary entry for June 14, 2024, reads:

"I had a tough day today.
Traffic was bad."
